#                               Pontificia Universidad Javeriana
#                       Tesis maestria en inteligencia Artificial
# Creado por Edwin Leonardo Silva Piracoca
# =============================================================

## Librerias

In [ ]:
!pip install PyPDF2
!pip install python-docx==0.8.11
!pip install openpyxl

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# ---------------------------------- COMBINANICON PDF WORD-----------------------------------------
import os
import re
import csv
import PyPDF2
import pandas as pd
from docx import Document
import logging

# Configurar el módulo de logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s', filename="error_log.txt")

# Función para extraer texto de un archivo PDF
def extract_pdf_text(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

# Función para extraer texto de un archivo Word
def extract_word_text(docx_path):
    try:
        doc = Document(docx_path)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return text
    except Exception as e:
        logging.error(f"Error al procesar el archivo Word {docx_path}. Detalles: {str(e)}")
        return ""

# Funcion extraer la seccion adecuada
def extract_section(content, start_pattern, end_pattern):
    section_match = re.search(start_pattern + r'.*?(?=\s' + end_pattern, content, re.DOTALL)
    section = section_match.group(0) if section_match else "No se encontró la sección."
    section = section.replace(start_pattern,"").replace(":", "").replace("cicl9","ciclo").replace("ciclo de 18 meses","ciclo").strip()

    return section

# Directorio donde se encuentran los archivos
folder_path = "C:\\Users\\57320\\Desktop\\Maestria\\Semestre 2023-3\\Tesis Maestria Agrosavia\\Base de datos AGROSAVIA\\Recopilación Recomendaciones 2017, 2018"


# Initialize an empty list to store data dictionaries
data_list = []

# Lista para guardar nombres de archivos problemáticos
problematic_files = []

# Recorrer archivos en la carpeta
for filename in os.listdir(folder_path):

    # Extraer el contenido basado en la extensión del archivo
    try:
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            content = extract_pdf_text(file_path)
        elif filename.endswith('.docx'):
            file_path = os.path.join(folder_path, filename)
            content = extract_word_text(file_path)

        else:
            continue
    except Exception as e:
        logging.error(f"Error al procesar el archivo {filename}. Detalles: {str(e)}")
        problematic_files.append(filename)
        continue

    # Create a corresponding .txt filename
    txt_filename = os.path.splitext(filename)[0] + ".txt"

    # Save the content to a .txt file
    with open(txt_filename, "w", encoding="utf-8") as txt_file:
            txt_file.write(content)

    # Buscar la sección del informe
    informe_match = re.search(r'INFORME N°.*?\d{4} -\d{2}-\d{2}', content, re.DOTALL)
    informe_section = informe_match.group(0) if informe_match else "No se encontró la sección del informe."
    usuario_section = extract_section(content, r'USUARIO', r'IDENTIFICACION)')
    ident_section = extract_section(content, r'IDENTIFICACION', r'NÚMERO DE LABORATORIO)')
    ident_section = ident_section.replace(" ", "").strip()
    num_lab_section = extract_section(content, r'NÚMERO DE LABORATORIO', r'FECHA)')
    fecha_section = extract_section(content, r'FECHA', r'DEPARTAMENTO)')
    departamento_section = extract_section(content, r'DEPARTAMENTO', r'MUNICIPIO)')
    municipio_section = extract_section(content, r'MUNICIPIO', r'VEREDA)')
    vereda_section = extract_section(content, r'VEREDA', r'FINCA)')
    finca_section = extract_section(content, r'FINCA', r'CULTIVO)')
    cultivo_section = extract_section(content, r'CULTIVO', r'EDAD)')
    edad_section = extract_section(content, r'EDAD', r'DIAGNÓSTICO DE LOS RESULTADOS DEL ANALISIS DE SUELO)')
    diagnostico_section = extract_section(content, r'DIAGNÓSTICO DE LOS RESULTADOS DEL ANALISIS DE SUELO', r'CANTIDAD DE NUTRIENTES)')

    nutrientes_section = extract_section(content, r'[kK]g/h?\s*a\s*?/?\s*?\s*?(ciclo|año|cosecha|cicl9)', r'Q?\s*u?\s*i?\s*e?\s*n\s+r?\s*e?\s*a?\s*l?\s*i?\s*z[oó]\s*)')
    if nutrientes_section == "No se encontró la sección.":
        #print("esta aca 3")
        nutrientes_section = extract_section(content, r'C  BORO\s*\s*\nKg/ha\s*\s*\n', r'Quien\s+realiz[oó]\s*)')
        if nutrientes_section == "No se encontró la sección.":
            #print("esta aca 3")
            nutrientes_section = extract_section(content, r'RECOMENDACIÓN DE FERTILIZACIÓN', r'Quien\s+realiz[oó]\s*)')



    nutrientes_section = re.sub(r'[^\d\s.]', '', nutrientes_section)

    # Dividir los números en una lista
    numbers_list = nutrientes_section.split()

    try:

        nitrogeno, fosforo, potasio, calcio, magnesio, azufre, hierro, manganeso, zinc, boro = numbers_list

        # Create a dictionary for the extracted data
        data = {
                "Informe": informe_section,
                "Usuario": usuario_section,
                "Identificacion": ident_section,
                "Número de Laboratorio": num_lab_section,
                "Fecha": fecha_section,
                "Departamento": departamento_section,
                "Municipio": municipio_section,
                "Vereda": vereda_section,
                "Finca": finca_section,
                "Cultivo": cultivo_section,
                "Edad": edad_section,
                "Nutrientes": nutrientes_section,
                "nitrogeno": nitrogeno,
                "fosforo": fosforo,
                "potasio": potasio,
                "calcio": calcio,
                "magnesio": magnesio,
                "azufre": azufre,
                "hierro": hierro,
                "manganeso": manganeso,
                "zinc": zinc,
                "boro": boro,
                "Diagnóstico": diagnostico_section,
            }

        # Append the data dictionary to the data_list
        data_list.append(data)
    except ValueError:
        logging.error(f"Error al procesar el archivo {filename}. La cantidad de valores en 'numbers_list' no coincide con lo esperado.")
        problematic_files.append(filename)

# Create a DataFrame from the list of data dictionaries
df = pd.DataFrame(data_list)

df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Informe                1 non-null      object
 1   Usuario                1 non-null      object
 2   Identificacion         1 non-null      object
 3   Número de Laboratorio  1 non-null      object
 4   Fecha                  1 non-null      object
 5   Departamento           1 non-null      object
 6   Municipio              1 non-null      object
 7   Vereda                 1 non-null      object
 8   Finca                  1 non-null      object
 9   Cultivo                1 non-null      object
 10  Edad                   1 non-null      object
 11  Nutrientes             1 non-null      object
 12  nitrogeno              1 non-null      object
 13  fosforo                1 non-null      object
 14  potasio                1 non-null      object
 15  calcio                 1 no

In [ ]:
df

,Informe,Usuario,Identificacion,Número de Laboratorio,Fecha,Departamento,Municipio,Vereda,Finca,Cultivo,...,nitrogeno,fosforo,potasio,calcio,magnesio,azufre,hierro,manganeso,zinc,boro
0,INFORME N° 2077-S16 ( 22531) Hernan Bejarano A...,Hernán Bejarano Acosta,036781,S16-22531,01-08-2016,Cundinamarca,Ubalá,San Juan,Trigales,Mejoramiento de Parcelas -Pastos,...,89,69,50,31,24,1.2,0.0,0.0,3.5,0.0
1,INFORME N° 1667-S16 ( 21132) Leonel Murillo C ...,Leonel Murillo C.,003602,S16-21132,08-06-2016,Meta,Guamal,El Danubio,La Envidia,Plátano hartón,...,161,58,270,58,32,75,0.0,0.0,3.1,0.9
2,INFORME N° 622-S16 (18045) Hector Julio Corral...,Héctor Julio Corrales,007624,S16-18045,30-03-2016,No Indica,No Indica,No Indica,No Indica,Caña Panelera,...,138,58,135,29,16,38,0.0,0.0,1.6,0.5
3,INFORME N° 1004-S16 (1 9174) Miguel Pacheco 2...,Miguel Pacheco,023015,S16-19174,26-04-2016,Córdoba,Monteria,Marac ayo,Media luna,Caña,...,150,80,60,29,0.0,27,0.0,0.0,0.0,0.0
4,INFORME N° 1006-S16 (1 9176) Carmen L Sereno ...,Carmen L. Serrano,019241,S16-19176,28-04-2016,Santand er,Puerto Wilches,Las pampas,Parcela,Palma africana,...,153,45,270,20,30,70,0.0,0.0,6.0,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,No se encontró la sección del informe.,Jaime López,020956,S16-16007,03-02-2016,Antioquia,Sonson,Manzanares Centro,Rincón de Antaño,Gulupa,...,53,46,75,22,12,28,0.0,0.0,1.2,0.5
3218,No se encontró la sección del informe.,Javier Quintero,025454,S16-21130,13-06-2016,Caldas,Victoria,Guyana,Manaos,Cacao,...,51,51,137,50,24,61,0.0,0.0,3.7,0.5
3219,No se encontró la sección del informe.,Ferney Carvallo,036253,S16-21591,23-06-2016,Cauca,Piamonte,San Jorge,El Regalo,Brachiaria humidicola,...,148,69,72,31,24,1.2,0.0,0.0,3.5,0.5
3220,No se encontró la sección del informe.,Luis Fernando Velásquez,1,S16-21950,08-07-2016,Antioquia,Turbo,El Esfuerzo,Dianita,Banano,...,223,46,165,0.0,0.0,26,0.0,1.2,4.1,0.5


In [ ]:
#problematic_files

In [ ]:
df_file_problem = pd.DataFrame(problematic_files)

df_file_problem

,0
0,INFORME N° 1126-S16 (19589) Luis Carlos Rivera...
1,INFORME N° 1320-S17 (19853) Fernando Díaz 2017...
2,INFORME N° 1322-S17 (19855) Fernando Díaz 2017...
3,INFORME N° 1327-S17 (19860) Ecotara Inversione...
4,INFORME N° 1332-S17 (20189) Smayd Ltda 2017-05...
...,...
110,Respuesta a oficio 60161101748.pdf
111,Suelos sulfatados ácidos.pdf
112,SUPLEMENTO AL INFORME N 348-S16 (17143) Alvaro...
113,SUPLEMENTO AL INFORME N° 3384-S17 (26484) Sand...


In [ ]:
# buscar un Usuario

result = df.loc[df['Usuario'] == 'Miguel Niño Landinez']
nutrientes_column = result['Nutrientes']
nutrientes_column

result

,Informe,Usuario,Identificacion,Número de Laboratorio,Fecha,Departamento,Municipio,Vereda,Finca,Cultivo,...,nitrogeno,fosforo,potasio,calcio,magnesio,azufre,hierro,manganeso,zinc,boro
578,No se encontró la sección del informe.,Miguel Niño Landinez,026108,S16-16324,09-02-2016,Santander,Hato,Hato negra,El chocho,Durazno,...,41,35,30,22,12,28,0.0,0.0,1.0,0.0
